In [1]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Define the CRNN class (must match exactly the original definition)
class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        # CNN component with dropout
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # (N, 64, 16, W/2)
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # (N, 128, 8, W/4)
            nn.Dropout2d(0.3),  # Dropout after second maxpool
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d((2,1), (2,1)),  # (N, 256, 4, W/4)
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d((2,1), (2,1)),  # (N, 512, 2, W/4)
            nn.Dropout2d(0.3),  # Dropout after fourth maxpool
            nn.Conv2d(512, 512, kernel_size=(2,1)),  # (N, 512, 1, W/4)
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )
        # LSTM with dropout between layers
        self.rnn = nn.LSTM(512, 256, num_layers=2, bidirectional=True, dropout=0.3)
        # Dropout before the fully connected layer
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, num_classes)  # 512 because bidirectional (256 * 2)

    def forward(self, x):
        # Pass through CNN
        x = self.cnn(x)  # (N, 512, 1, W/4)
        x = x.squeeze(2)  # (N, 512, W/4)
        x = x.permute(2, 0, 1)  # (W/4, N, 512) for LSTM
        # Pass through LSTM
        x, _ = self.rnn(x)  # (W/4, N, 512)
        # Apply dropout before FC
        x = self.dropout(x)
        # Fully connected layer for classification
        x = self.fc(x)  # (W/4, N, num_classes)
        return x

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model
model = CRNN(num_classes=11)
model.to(device)

# Load the saved model
best_model_path = 'saved_models/best_crnn_model.pth' # original : saved_models/best_crnn_model.pth
checkpoint = torch.load(best_model_path, map_location=device)  # map_location ensures compatibility
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Print some info about the loaded model
print(f"Loaded model from epoch {checkpoint['epoch']} with Val Acc: {checkpoint['val_accuracy']:.2f}%")

# Define the transform (same as val_test_transform from training)
val_test_transform = transforms.Compose([
    transforms.Resize((32, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Prediction function (same as Cell 6)
def predict_register_number(model, image_path, device, transform):
    image = Image.open(image_path).convert('L')
    image = transform(image)
    image = image.unsqueeze(0)  # (1, 1, 32, 256)
    with torch.no_grad():
        image = image.to(device)
        output = model(image)
        output = output.squeeze(1)
        output = output.softmax(1).argmax(1)
        seq = output.cpu().numpy()
        prev = -1
        result = []
        for s in seq:
            if s != 0 and s != prev:
                result.append(s - 1)
            prev = s
    return ''.join(map(str, result))

# Test with an image
image_path = 's8.jpeg'  # Replace with your image path
predicted_number = predict_register_number(model, image_path, device, val_test_transform)
print(f"Predicted Register Number: {predicted_number}")

Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 212220040011


In [ ]:
Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 212220040011

In [ ]:
# Loaded model from epoch 30 with Val Acc: 89.53%
# Predicted Register Number: 212223240065


# Loaded model from epoch 41 with Val Acc: 90.47%
# Predicted Register Number: 212223240065

# Loaded model from epoch 62 with Val Acc: 93.83%
# Predicted Register Number: 212223240065



## saved_model_temp : Loaded model from epoch 68 with Val Acc: 93.27%


In [ ]:
Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 212220230047
    
Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 212220230050
    
Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 212220230049

    
Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 21222324004
    
Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 212223230135

Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 21222243005

Loaded model from epoch 62 with Val Acc: 93.83%
Predicted Register Number: 212222230236
